# Feedback from previous weeks and other and hints

1. Be careful about your environment remembering variables. Make sure your code works in a new _clean_ environment. In Colab: `Runtime`->`restart Runtime`, in Anaconda's Jupyter: `Kernel`->`Restart`.
2. Keep the file names when saving to GitHub. It's always possible to go back to a previous version, you are not losing anything.
3. Run all the cells before saving to GitHub so the output is saved.
4. Graphs without labels (or units when appropriate) are not worth any point.
5. Do put in sufficient explanatory comments in your code.

For this week you can use these imports:

In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Introduction

In this worksheet we’ll have a further look at `curve_fit()`, in particular in the presence of noisy data.

We will look at simulating the analysis of a continuous gravitational-wave signal from deformed Neutron Stars of the kind analysed in [this paper](https://arxiv.org/pdf/2107.00600.pdf). While we will use a simplified signal, those objects do emit sinusoidal gravitational waves.

To illustrate the effect of the noise, it can be helpful to make animation. Look at the snippet bellow, you will have to tranform it for some of the exercises.

In [5]:
# Some imports for animating.
from matplotlib.animation import FuncAnimation
from IPython import display # This is specific to IPython, upon which jupyter notebooks are built.

In [6]:
# Creating an empty figure
fig = plt.figure()
ax = plt.axes()

# Initialisation of the plot element `line` as empty:
line, = ax.plot([],label='data',color='red')
# We could in general have more than one plot element.

x=np.linspace(0,2*np.pi,100)

# Setting axes so they don't move from frame to frame
ax.set_xlim(0,2*np.pi)
ax.set_ylim(-1.1,1.1)

# The anination function, which gets run for each frame, with a
# different frame number. It has to accept as input a single
# frame number, and replace the data for each frame number
def animate(frame_num):

    # This function updated the data of the line element.
    # In this example we have a sine wave, but this could
    # be anything that depends on the frame number.
    line.set_data(x,np.sin(x+frame_num/100))
    return line

# Animation function, for `frames` number of frame, with `interval` ms
# between frames.
anim=FuncAnimation(fig,animate,frames=100, interval=20)

# And display
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

# Exercises
This must be marked before you leave the lab. Mark weighting is in brackets.
**Save your work to GitHub after having run all cells with `Runtime` -> `Restart and run all`. And do not change the notebook's filename.** Do add comments to your code, you'll lose points if your code is hard to understand. Graphs without labels (or units when appropriate) are not worth any point.

## Exercise 0

See the bottom of the worksheet for Exercise 0, solved on video available on LearningCentral.

## Exercise 1

1. [2] Write a function that returns a sinusoid signal, in the form that can be used by curve_fit(). It should take as input an array of time values, and be parametrised by an amplitude, a frequency and a phase. You should be able to reuse the function you wrote for Exercise 1 in week 2.

    Plot the resulting signal from calling that function with unit amplitude, unit frequency, zero phase over an array of time from 0 to 10, with at least 100 points. In this example, we are using arbitrary units where all the quantities become numbers close to 1. This is a common technique to avoid large dynamic ranges in the numbers used by the program.

2. [2] The signal from 1) with unit amplitude, unit frequency and zero phase will be our reference signal. We are now trying to see how much noise can be added until we have troubles recovering it. This could for instance help set an experiment's requirement.

    Create a function that returns random Gaussian noise, centered around zero, and with the same number of points as in an array of time values. The function should take as input the standard deviation of the noise, and something to tell how many points are needed: either 1) the number of points to generate directly, or 2) an array to take the length of.

    Plot the noise obtained from that function with unit standard deviation on top of the signal from the question above, as a function of the same time array.

3. [2] Putting things together, create a function with the following call signature:
   
   ```python
   gen_data(t,signal,sigma_noise=1)
   ```
   Which returns the sum of the signal and noise generated with `sigma_noise` as its standard deviation, using the functions you wrote in question 1) and in question 2). Plot the result  few times (i.e. for a few different random noises), with the reference signal.

4. [4] We will now run `curve_fit()` on the data we created, and see how well we can recover our reference signal. Create some data by using `gen_data()` to generate data with our reference signal added to random Gaussian noise of unit standard deviation.

    First, plot this data with the reference signal and a fitting signal from your sinusoid function from Exercise 1 that is a bit offset from the true signal values.

    Then, run `curve_fit()` on the data with the signal model. Plot and print the result.

5. [2] From the amplitude $a$ of the signal and its frequency $f$, we can compute the quantity:
$\frac{a}{f^2}$. Which, if we can find out the distance between us and the Neutron Star at the source of this signal, is a useful quantity to infer the star's mass quadrupole moment (and its moment of inertia).

    Use the `uncertainties` package to propagate the uncertainties measured by `curve_fit()` into the uncertainties for $\frac{a}{f^2}$.

6. [4] Repeat the process above (create noise, create data=signal+noise, fit the data) with different `sigma_noise` values. Try to see what values are reached before the uncertainty of the fit obtained with `curve_fit()` becomes very large. Illustrate this with an animation, choosing appropriate values.

In [7]:
hello


NameError: name 'hello' is not defined

## Exercise 2

Gravitational waves have two polarisations, usually called $h_+(t)$ and $h_{\times}(t)$. Is is convenient to write a gravitational wave as a complex time series $h(t)=h_+(t)+ih_{\times}(t)$, with the polarisation angle then the argument of the complex numbers.

1. [1] Write a function that, from an amplitude $A$, a frequency $f$ and a phase $\phi$ generates the complex time series:

$$
h(t)=A\,\exp(i(2\pi f t + \phi))
$$

2. [1] Make a 3D plot of that function for $A=1,f=1,\phi=0$, using the axes $(\text{Real}(h(t)),\text{Imag}(h(t)),t)$. To make a 3D plot, of, say, the arrays `x`, `y` and `z` you can use:

```python
from mpl_toolkits import mplot3d
ax = plt.axes(projection='3d')
ax.plot3D(x, y, z)
```

3. [2] Create an animation of the plot by varying the phase. You have to make the animation code from above work with a 3D figure. You can replace `line.set_data()` with `line.set_data_3d()`.

## Exercise 0

[0] When we know the data we are trying to fit with `curve_fit()` is more noisy in some places, we can use [`curve_fit()`'s weighted least squares fitting](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html). To illustrate this, create a fitting function for a [Lorentzian line shape function](https://en.wikipedia.org/wiki/Spectral_line_shape#Lorentzian). This is often used in gravitational-wave detectors to model lines in the noise spectrum.

$$
line(f)=\frac{A\,\gamma^2}{\gamma^2+(f-f_0)^2}
$$

Over a frequency array from 1 to 20 and at least 100 frequencies, generate:
- a Lorentzian signal with $A=3,\gamma=5,f_0=12$ (arbitrary units)
- Gaussian noise with a standard deviation of $1$ everywhere except for a window from $f=10$ to $f=14$ where the noise is higher, with a standard deviation of $5$.
- perform the fit with and without weights (the `sigma` argument in `curve_fit()`), running the program until you see a good illustation of weighted least squares fitting.

(this exercise is for demonstration purposes and won't be marked)